In [33]:
# import section

import pandas as pd
pd.set_option('display.max_columns', 500)
import os
import numpy as np
import matplotlib.pyplot as plt 
from collections import Counter
from sklearn.preprocessing import OneHotEncoder, Normalizer, Binarizer
from sklearn.compose import make_column_transformer
import seaborn as sns

In [34]:
data_path = './data/'
train_accounts = pd.read_csv(data_path + 'train_accounts.csv')
test_accounts = pd.read_csv(data_path + 'test_accounts.csv')

c:\users\yama\docume~1\datapr~1\dh_yyr\venv\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
train_accounts.head()

,account_id,account_name,created_at,plan_id,trial_start,churn_date,churn_reason,time_diff,region,country,subscription_started_at,paying,has_logo,device,os,browser,collection_21_days,company_size,payment_currency,max_team_size,min_team_size,industry,utm_cluster_id,mrr,user_goal,user_description,team_size,lead_score
0,1.0,"Gardner, Barron and Keller",2019-01-01 00:01:15,NaN,2019-01-01 00:01:15,NaN,NaN,11.0,New South Wales,AU,NaN,0,1,desktop,windows,microsoft edge,0,NaN,AUD,5.0,2.0,NaN,orders,NaN,NaN,NaN,NaN,0
1,2.0,Dunn Ltd,2019-01-01 00:01:52,NaN,2019-01-01 00:01:52,NaN,NaN,NaN,New Jersey,US,NaN,0,1,mobile,ios,NaN,0,NaN,USD,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3.0,Boone Inc,2019-01-01 00:03:12,NaN,2019-01-01 00:03:12,NaN,NaN,-6.0,Louisiana,US,NaN,0,1,desktop,windows,chrome,0,NaN,USD,1.0,1.0,Other,todos,NaN,NaN,NaN,1,0
3,4.0,"Christian, Carroll and Davis",2019-01-01 00:04:11,NaN,2019-01-01 00:04:11,NaN,NaN,NaN,Tel Aviv,IL,NaN,0,1,mobile,android,NaN,0,NaN,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5.0,Brooks-Oliver,2019-01-01 00:04:21,NaN,2019-04-04 11:09:12,NaN,NaN,-5.0,North Carolina,US,NaN,0,1,desktop,chrome_os,chrome,0,NaN,USD,1.0,1.0,Design,todos,NaN,NaN,NaN,1,0


In [36]:
test_accounts.head()

,account_id,account_name,created_at,plan_id,trial_start,churn_date,churn_reason,time_diff,region,country,subscription_started_at,paying,has_logo,device,os,browser,collection_21_days,company_size,payment_currency,max_team_size,min_team_size,industry,utm_cluster_id,mrr,user_goal,user_description,team_size
0,7.0,Phillips-Mendoza,2019-01-01 00:04:31,NaN,2019-01-01 00:04:31,NaN,NaN,2.00,Tel Aviv,IL,NaN,0,1,desktop,windows,chrome,0,NaN,USD,1.0,1.0,Business Owner,orders,NaN,NaN,NaN,1-1
1,44.0,Peterson-Torres,2019-01-01 00:34:55,NaN,2019-01-01 00:34:55,NaN,NaN,-5.00,Quintana Roo,MX,NaN,0,1,desktop,mac,chrome,0,NaN,USD,5.0,2.0,Business Development,todos,NaN,NaN,NaN,2-5
2,45.0,Small-Robinson,2019-01-01 00:35:38,NaN,2019-01-01 00:35:38,NaN,NaN,13.75,Central,FJ,NaN,0,1,desktop,windows,chrome,0,NaN,USD,1.0,1.0,Business Owner,client_projects,NaN,NaN,NaN,1-1
3,46.0,"Herrera, Smith and Singh",2019-01-01 00:36:19,NaN,2019-01-01 00:36:19,NaN,NaN,-8.00,Texas,US,NaN,0,1,desktop,windows,microsoft edge,0,NaN,USD,1.0,1.0,Administrative,client_projects,NaN,NaN,NaN,1
4,53.0,Lopez LLC,2019-01-01 00:39:39,NaN,2019-02-07 21:36:57,NaN,NaN,-5.00,South Carolina,US,NaN,0,1,desktop,mac,safari,0,NaN,USD,1.0,1.0,Business Owner,todos,NaN,NaN,NaN,1-1


In [39]:
# We map our features into different types
categorical_features = ['os']

normalized_features = ['collection_21_days']

binary_features = ['plan_id']

untouched_features = ['paying']

target = ['lead_score']

# And create a column transformer to handle the manipulation for us
preprocess = make_column_transformer(
    (OneHotEncoder(), categorical_features),
    (Normalizer(), normalized_features),
    (Binarizer(), binary_features)
)

In [48]:
# Getting only the relevant features from the dataset
dataset_train = train_accounts[categorical_features + normalized_features + binary_features + untouched_features + target]
dataset_test = test_accounts[categorical_features + normalized_features + binary_features + untouched_features]

# Filling empty values with default values 
def fill_empty_values(dataset):
    dataset.loc[:,categorical_features] = dataset[categorical_features].fillna('')
    dataset.loc[:,normalized_features + binary_features + untouched_features] = dataset[normalized_features + binary_features + untouched_features].fillna(0)
    return dataset

dataset_train = fill_empty_values(dataset_train)
dataset_test = fill_empty_values(dataset_test)

In [58]:
# We fit our column transformer on both the train and the test sets
concatenated = pd.concat([dataset_train, dataset_test])
preprocess.fit(concatenated)

# We use transform to finally manipulate the features of our training set
dataset_train = dataset_train[concatenated.columns]
x = preprocess.transform(dataset_train)

# Seperating the label
y = dataset_train.pop('lead_score')

c:\users\yama\docume~1\datapr~1\dh_yyr\venv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
